# Video Lecture 

## Steps for learning - 
1. watch lecture
2. read chapter
3. replicate the chapter notebook
4. try it on new dataset-

** can connect git to VS code, jupyter (everything!)

## some ideas popped up
1. stages of rotten banana classification - don't eat, can eat, wait to eat 
2. dal classifier - I suck at identifying the type (kind of bad for me according to Indian mom)
3. marvel vs dc movie poster classifier
4. face shape classifier and recommend hair styles 

## coding tips
1. we can use latex typset by enclosing in $ for plot titles and printing
2. functools-> partial, to fix the parameter values for your function

## modelling tips
1. start with basic archeitecture and experiment with data related things for faster results, last thing would be to change the archeitecture
2. try to think of trade off between speed and accuracy

## cool sites
1. matrixmultiplication.xyz

- sum of relus is like splines in linear regression
- 

